In [1]:
import torch 
import torch.nn as nn 
import numpy as np
from collections import OrderedDict

from utils.losses import protop_loss
from utils.sample_parameters import SingleGenerator, HiddenAndKernelGenerator, MonotonicGenerator
from utils.sample_parameters import ParamGenerators
from models.ProtICU import ProtICU
from utils.train_n_test import TrainTest

In [2]:
train, val, test = torch.load('data/in-hospital-mortality/tensors.pkl')

In [3]:
# Calculating class weights
perc_mort = np.concatenate((train[1], val[1], test[1])).mean()
class_weights = torch.Tensor([perc_mort, 1-perc_mort])

In [4]:
generators_dict = {
    # HYPERMARAMETER RANGES
    'BATCH_SIZE': SingleGenerator([128, 256, 512]),
    'EPOCHS': SingleGenerator([20, 40, 50]),
    'OPTIMIZER': SingleGenerator([torch.optim.Adam]),
    'LEARNING_RATE': SingleGenerator([1e-5, 5e-5, 1e-4, 5e-4]),
    'LOSS': SingleGenerator([protop_loss(class_weights, .5 , .5), protop_loss(class_weights, 1, 1),
                             protop_loss(class_weights, .1 , .1), protop_loss(class_weights, 1 , .5),
                             protop_loss(class_weights, .5 , .1), protop_loss(class_weights, .3 , .1)]),
    'EARLY_STOPPING': SingleGenerator([True, False]),
    'PATIENCE': SingleGenerator(list(range(2,4))),
    'MIN_DELTA': SingleGenerator([5e-5, 1e-4, 5e-4, 1e-3, 5e-3]), 

    # NETWORK PARAMETER RANGES
    'HIDDEN_AND_KERNEL_SIZES': HiddenAndKernelGenerator(range(2,4), [128, 256, 512], [3, 5, 7, 9], 
                               ascending=(True, False)),
    'MAXPOOL': SingleGenerator([2]),
    'OBO_SIZES': MonotonicGenerator(range(1,3), [256, 512], ascending=False),
    'PROTOTYPE_NUM': SingleGenerator([10,20]),
    'DROPOUT': SingleGenerator(np.arange(8)/10), 
    'PROTO_ACTIVATION': SingleGenerator(['log', 'linear'])
} # each of these params are sampled INDEPENDENTLY of one another

In [5]:
fixed_gen = {
    # HYPERMARAMETER RANGES
    'BATCH_SIZE': SingleGenerator([256]),
    'EPOCHS': SingleGenerator([20]),
    'OPTIMIZER': SingleGenerator([torch.optim.Adam]),
    'LEARNING_RATE': SingleGenerator([1e-5]),
    'LOSS': SingleGenerator([protop_loss(class_weights, 1, 1)]),
    'EARLY_STOPPING': SingleGenerator([True]),
    'PATIENCE': SingleGenerator(list(range(1,2))),
    'MIN_DELTA': SingleGenerator([1e-4]), 

    # NETWORK PARAMETER RANGES
    'HIDDEN_AND_KERNEL_SIZES': HiddenAndKernelGenerator(range(1,2), [256], [5], 
                               ascending=(True, False)),
    'MAXPOOL': SingleGenerator([2]),
    'OBO_SIZES': MonotonicGenerator(range(1,2), [128], ascending=False),
    'PROTOTYPE_NUM': SingleGenerator([20]),
    'DROPOUT': SingleGenerator([0.2]),
    'PROTO_ACTIVATION': SingleGenerator(['log'])
}

In [6]:
N = 1
gen = ParamGenerators(generators_dict)
gen = ParamGenerators(fixed_gen)
param_samples = gen.sample(N)

In [7]:
stats = []
for i in np.arange(N):
    print(param_samples[i])
    param_samples[i]['EPOCHS']
    push_epochs = [param_samples[0]['EPOCHS'] -1] #list(range(5,param_samples[0]['EPOCHS'],2))
    run = TrainTest(ProtICU, (train, val, test), param_samples[i], push_epochs=push_epochs)
    run.train()
    stats.append(run.test())

  0%|          | 0/57 [00:00<?, ?it/s]

{'BATCH_SIZE': 256, 'EPOCHS': 20, 'OPTIMIZER': <class 'torch.optim.adam.Adam'>, 'LEARNING_RATE': 1e-05, 'LOSS': protop_loss_1_1, 'EARLY_STOPPING': True, 'PATIENCE': 1, 'MIN_DELTA': 0.0001, 'MAXPOOL': 2, 'OBO_SIZES': array([128]), 'PROTOTYPE_NUM': 20, 'DROPOUT': 0.2, 'PROTO_ACTIVATION': 'log', 'HIDDEN_SIZES': array([256]), 'KERNEL_SIZES': array([5])}
tensor([[0.1580, 0.1663, 0.1715,  ..., 0.1591, 0.1681, 0.1714],
        [0.1578, 0.1629, 0.1714,  ..., 0.1591, 0.1666, 0.1677],
        [0.1563, 0.1648, 0.1719,  ..., 0.1598, 0.1703, 0.1712],
        ...,
        [0.1572, 0.1693, 0.1727,  ..., 0.1583, 0.1678, 0.1674],
        [0.1598, 0.1649, 0.1740,  ..., 0.1581, 0.1702, 0.1723],
        [0.1603, 0.1655, 0.1727,  ..., 0.1593, 0.1685, 0.1728]],
       grad_fn=<LogBackward>)


  2%|▏         | 1/57 [00:00<00:26,  2.09it/s]

tensor([[0.1602, 0.1672, 0.1729,  ..., 0.1600, 0.1699, 0.1715],
        [0.1560, 0.1649, 0.1743,  ..., 0.1583, 0.1677, 0.1703],
        [0.1554, 0.1665, 0.1701,  ..., 0.1568, 0.1663, 0.1646],
        ...,
        [0.1588, 0.1674, 0.1732,  ..., 0.1590, 0.1666, 0.1657],
        [0.1601, 0.1661, 0.1746,  ..., 0.1609, 0.1718, 0.1720],
        [0.1575, 0.1642, 0.1741,  ..., 0.1593, 0.1677, 0.1716]],
       grad_fn=<LogBackward>)


  4%|▎         | 2/57 [00:01<00:27,  2.00it/s]

...

[tensor(5.7473), tensor(5.3654), tensor(5.1902), tensor(5.4109), tensor(5.1244), tensor(5.1691), tensor(5.6235), tensor(5.2234), tensor(5.6081), tensor(3.2698), tensor(5.6052), tensor(5.4445), tensor(5.7171), tensor(5.5838), tensor(5.8420), tensor(5.5210), tensor(5.2636), tensor(5.7620), tensor(5.4826), tensor(5.5038)]


 98%|█████████▊| 56/57 [00:18<00:00,  3.12it/s]

[tensor(5.7473), tensor(5.3654), tensor(5.1902), tensor(5.4109), tensor(5.1244), tensor(5.1691), tensor(5.6235), tensor(5.2234), tensor(5.6081), tensor(3.2698), tensor(5.6052), tensor(5.4445), tensor(5.7171), tensor(5.5838), tensor(5.8420), tensor(5.5210), tensor(5.2636), tensor(5.7620), tensor(5.4826), tensor(5.5038)]


100%|██████████| 57/57 [00:18<00:00,  3.09it/s]


[tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128), tensor(128)]
Epoch: 19, train_loss: -0.8884828090667725, valid_loss: -0.8602588772773743
Parameter containing:
tensor([[ 0.9998,  0.9998,  0.9998,  0.9998,  0.9999,  0.9997,  1.0000,  0.9997,
          0.9997,  1.0004, -0.5002, -0.5003, -0.5003, -0.5002, -0.5003, -0.5002,
         -0.5001, -0.5004, -0.5002, -0.5003],
        [-0.4998, -0.4998, -0.4998, -0.4998, -0.4999, -0.4997, -0.5000, -0.4997,
         -0.4997, -0.5004,  1.0002,  1.0003,  1.0003,  1.0002,  1.0003,  1.0002,
          1.0001,  1.0004,  1.0002,  1.0003]], requires_grad=True)
tensor([[0.3266, 0.3047, 0.2609,  ..., 0.2714, 0.2553, 0.2548],
        [0.3812, 0.3181, 0.3391,  ..., 0.4155, 0.3493, 0.3148],
        [0.4078, 0.3192, 0.3648,  ..., 0.3984, 0.3528, 0.3762],
        ...,

In [57]:
# Check 
check = test[0][:1]
out, min_dis = run.model(check)
value, proto_id = torch.min(min_dis, dim=1)


tensor([[0.3266, 0.3047, 0.2609, 0.2267, 0.2845, 0.2609, 0.2763, 0.3001, 0.2796,
         0.3776, 0.2701, 0.2667, 0.3372, 0.3004, 0.2598, 0.2597, 0.2553, 0.2714,
         0.2553, 0.2548]], grad_fn=<LogBackward>)


In [26]:
# make params_sample
for i in np.arange(N):
    for key,val in stats[i].items():
        param_samples[i][key] = val
        
# saving 
torch.save(param_samples, 'results/Proto_experiment_no_push_N'+str(N)+'.pkl')

# getting pd 
import pandas as pd 
df = pd.DataFrame(param_samples)

In [28]:
df.sort_values('auroc', ascending=False)

,BATCH_SIZE,EPOCHS,OPTIMIZER,LEARNING_RATE,LOSS,EARLY_STOPPING,PATIENCE,MIN_DELTA,MAXPOOL,OBO_SIZES,PROTOTYPE_NUM,DROPOUT,HIDDEN_SIZES,KERNEL_SIZES,epoch_stopped,auroc,auprc,acc,f1
4,256,50,<class 'torch.optim.adam.Adam'>,0.00001,protop_loss_0.3_0.1,True,3,0.00050,2,"[256, 256]",20,0.5,"[128, 256, 512]","[7, 3, 3]",NaN,0.792926,0.404332,0.777673,0.440476
1,512,40,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_1_0.5,True,2,0.00010,2,"[256, 256]",10,0.3,"[128, 256, 512]","[7, 5, 3]",16.0,0.779959,0.433967,0.832545,0.476331
8,128,40,<class 'torch.optim.adam.Adam'>,0.00005,protop_loss_0.3_0.1,True,3,0.00500,2,[256],20,0.6,"[128, 256]","[9, 7]",23.0,0.753227,0.404382,0.839877,0.469020
2,256,50,<class 'torch.optim.adam.Adam'>,0.00010,protop_loss_0.5_0.1,True,3,0.00010,2,"[512, 256]",20,0.2,"[128, 256, 512]","[7, 5, 3]",24.0,0.749045,0.399017,0.828997,0.480976
3,256,40,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_0.3_0.1,True,3,0.00050,2,"[512, 256]",20,0.2,"[128, 256, 512]","[9, 7, 3]",18.0,0.741501,0.369021,0.790445,0.471990
5,512,40,<class 'torch.optim.adam.Adam'>,0.00005,protop_loss_0.1_0.1,True,2,0.00005,2,[512],10,0.1,"[256, 512]","[7, 7]",8.0,0.719210,0.289019,0.843425,0.032164
7,128,20,<class 'torch.optim.adam.Adam'>,0.00001,protop_loss_1_0.5,True,2,0.00050,2,[512],10,0.1,"[256, 512]","[7, 3]",12.0,0.647657,0.230045,0.838931,0.107471
0,512,40,<class 'torch.optim.adam.Adam'>,0.00001,protop_loss_1_1,True,2,0.00010,2,"[512, 256]",20,0.3,"[128, 256, 512]","[5, 5, 5]",8.0,0.618752,0.220073,0.846500,0.026987
6,256,50,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_0.1_0.1,True,3,0.00500,2,[256],20,0.3,"[128, 256]","[9, 7]",9.0,0.536037,0.143266,0.618496,0.021831
9,128,20,<class 'torch.optim.adam.Adam'>,0.00001,protop_loss_0.1_0.1,True,3,0.00100,2,[512],10,0.1,"[128, 256]","[9, 3]",9.0,0.464464,0.130880,0.296831,0.247913


In [8]:
model = ProtICU(test[0].shape, 2, [64, 128], [5,3], 2, [256, 128], [0,0,0,0,0,1,1,1,1,1], .2)
out, min_dis = model(test[0][:14])

In [ ]:
check = torch.load('results/Proto_experiment_nopush_N30.pkl')